In [28]:
import pandas as pd
from pandas import DataFrame, Series
data = pd.read_csv('20220314_d3Neuron_pooled_DDA.csv')
data

/tmp/ipykernel_57/1277650556.py:3: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('20220314_d3Neuron_pooled_DDA.csv')


,Checked,Confidence,Annotated Sequence,Modifications,Contaminant,Qvality PEP,Qvality q-value,# Protein Groups,# Proteins,# PSMs,...,Abundance: F1: Sample,Quan Info,Found in Sample: [S1] F1: Sample,Confidence (by Search Engine): Sequest HT,m/z [Da] (by Search Engine): Sequest HT,RT [min] (by Search Engine): Sequest HT,Percolator q-Value (by Search Engine): Sequest HT,Percolator PEP (by Search Engine): Sequest HT,XCorr (by Search Engine): Sequest HT,Top Apex RT [min]
0,False,High,[R].AAAAAAAAAAAAAAAGAGAGAK.[Q],NaN,False,2.204580e-09,0.000043,1,1,1,...,1.254141e+06,NaN,High,High,532.95557,73.9651,0.000031,1.034000e-09,4.10,73.95
1,False,High,[KR].NVLIFDLGGGTFDVSILTIEDGIFEVK.[S],NaN,False,7.150400e-06,0.000043,2,2,8,...,1.425142e+06,Shared,High,High,971.18768,163.2907,0.000031,3.533000e-06,2.18,163.16
2,False,High,[R].KALEESESSQLISPPLAQAIR.[D],NaN,False,3.124650e-06,0.000043,1,1,1,...,NaN,No Quan Values,High,High,756.41681,82.7177,0.000031,1.538000e-06,2.79,NaN
3,False,High,[R].NVLLDPQLVPGGGASEMAVAHALTEK.[S],1xOxidation [M17],False,4.429180e-06,0.000043,1,1,2,...,1.276454e+07,NaN,High,High,878.46161,102.9777,0.000031,2.187000e-06,2.49,103.10
4,False,High,[R].VLTLSEDSPYETLHSFISNAVAPFFK.[S],NaN,False,1.040140e-06,0.000043,1,1,4,...,2.059339e+06,NaN,High,High,971.49982,145.1595,0.000031,5.089000e-07,1.99,145.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34386,False,Medium,[K].SAGILTLPIMSR.[N],1xOxidation [M10],False,7.842370e-01,0.049924,1,1,1,...,NaN,No Quan Values,Medium,Medium,637.86249,91.7406,0.026610,5.968000e-01,0.91,NaN
34387,False,Medium,[K].SAGWTLATLGEQGLPHQSHRQR.[S],NaN,False,7.842370e-01,0.049924,1,1,1,...,NaN,No Quan Values,Medium,Medium,810.75085,146.1933,0.026560,5.960000e-01,0.94,NaN
34388,False,Medium,[-].MPVLGVASK.[L],1xOxidation [M1],False,7.842370e-01,0.049924,1,1,1,...,8.343970e+05,NaN,Medium,Medium,459.26089,104.3259,0.026650,5.978000e-01,1.21,104.38
34389,False,Medium,[K].EKPKEVAENQQNQSSDPEEEKGSQPPPAAESQSSLR.[R],NaN,False,7.842370e-01,0.049924,1,1,1,...,NaN,No Quan Values,Medium,Medium,984.72211,32.4376,0.026580,5.966000e-01,0.92,NaN


In [29]:
import numpy as np
from sklearn.neighbors import LocalOutlierFactor

In [30]:
#The column number may change based on your own data file. 
x = data.iloc[:,19] #m/z
y = data.iloc[:,20] #RT
z = data.iloc[:,4] #Contaminant
h = data.iloc[:,10] #protein
l = data.iloc[:,15]#abundance
m = data.iloc[:,2]#peptide sequence

In [31]:
m

0                      [R].AAAAAAAAAAAAAAAGAGAGAK.[Q]
1                [KR].NVLIFDLGGGTFDVSILTIEDGIFEVK.[S]
2                       [R].KALEESESSQLISPPLAQAIR.[D]
3                  [R].NVLLDPQLVPGGGASEMAVAHALTEK.[S]
4                  [R].VLTLSEDSPYETLHSFISNAVAPFFK.[S]
                             ...                     
34386                            [K].SAGILTLPIMSR.[N]
34387                  [K].SAGWTLATLGEQGLPHQSHRQR.[S]
34388                               [-].MPVLGVASK.[L]
34389    [K].EKPKEVAENQQNQSSDPEEEKGSQPPPAAESQSSLR.[R]
34390                             [K].EPPAKVPEVPK.[K]
Name: Annotated Sequence, Length: 34391, dtype: object

In [32]:
type(x)

pandas.core.series.Series

In [33]:
vector_list=[]
for i in range(0,34391):
    vector=[x[i],y[i]]
    vector_list.append(vector)

In [34]:
# Here is to adjust n_neighbors parameter
clf = LocalOutlierFactor(n_neighbors=5)
clf.fit_predict(vector_list)
result = clf.fit_predict(vector_list)

In [35]:
result

array([1, 1, 1, ..., 1, 1, 1])

In [36]:
clf.negative_outlier_factor_

array([-1.01443878, -1.01513442, -1.06982843, ..., -1.10546337,
       -1.06367114, -0.98638418])

In [37]:
#Briefly check how mnay outliers in your data.
count = 0
for i in result:
    if i == -1:
        count +=1
count

136

In [38]:
import csv
last_row=len(data)
with open('052225_d3 neuron_n=5.csv','w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['m/z','RT','fit_predict','negative','abundance','proteins','contamination','peptide sequence'])
    for a in range(0,last_row):
        c1=x[a]
        c2=y[a]
        c3=result[a]
        c4=clf.negative_outlier_factor_[a]
        c5=l[a]
        c6=h[a]
        c7=z[a]
        c8=m[a]
        writer.writerow([c1,c2,c3,c4,c5,c6,c7,c8])